# Tree-Based Methods & Ensembling Techniques (Notebook Development In Progress...)

## Intro

<br /><br />

### Motivation

Most of the models we've learned so far (linear/logistic regression, svm) and models we will learn in the upcoming classes (neural networks) involve <b>gradient-based weight optimization (using SGD).</b>

In this class we will go through tree-based models, known as decision trees, that are different in a sense that they <b>do not require</b> weight optimization via SGD. Besides, tree-based models are <b>nonlinear</b>.

Besides, we will learn about model ensembling - powerful technique in ML of combining multiple models in different ways in order to get much powerful models. Particularly, we will examine two most popular and widely used ways of decision tree ensembling - <b>Random Forests</b> and <b>Gradient Boosting</b>.

As usual, there will be accompanying examples in Scikit-Learn.


### Learning Objectives

At the end of this class, listeners will be able to:

<ul>
    <li>Understand key concepts of decision trees and their ensembles.</li>
    <li>Understand intuition behind different types of model ensembling.</li>
    <li>Implement simple models using Scikit-Learn and Python.</li>
</ul>

### Reading Material (ordered by difficulty)

Decision Trees (for Classification and Regression)

<ul>
    <li>https://www.youtube.com/watch?v=GZuweldJWrM&list=PLnZuxOufsXnvftwTB1HL6mel1V32w0ThI&index=20 - Bloomberg's ML lecture explaining decision trees for classification and regression. (<b>recommended prerequisite for this class</b>)</li>
    <li>https://scikit-learn.org/stable/modules/tree.html - Good explanation of Tree methods with Cons and Pros and practical tips</li>
    <li>T. Hastie, R. Tibshirani and J. Friedman -  Elements of Statistical Learning, SECTION 9.2 - In-depth explanation of theory behind classification/regression trees.</li>
</ul>

Ensembling

<ul>
    <li>
        https://www.youtube.com/watch?v=f2S4hVs-ESw&list=PLqIj-nyfGu55TQpzjv_kwZcz3pJSW9bnH&index=22 - Bloomberg's ML lecture explaining tree ensembles. (<b>recommended prerequisite for this class</b>)
    </li>
    <li>http://blog.kaggle.com/2017/01/23/a-kaggle-master-explains-gradient-boosting/</li>
    <li>https://statweb.stanford.edu/~jhf/ftp/trebst.pdf - In-depth theory of Gradient Boosting (Advanced)</li>
</ul>

<br /><br />

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_boston

%matplotlib inline

For the first part of this class we gonna use toy dataset that comes with Scikit-Learn library. The dataset is about predicting house prices in Boston using 13 continuous features: https://scikit-learn.org/stable/datasets/index.html

These are the features of the data:

CRIM - per capita crime rate by town

ZN - proportion of residential land zoned for lots over 25,000 sq.ft.

INDUS - proportion of non-retail business acres per town

CHAS - Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)

NOX - nitric oxides concentration (parts per 10 million)

RM - average number of rooms per dwelling

AGE - proportion of owner-occupied units built prior to 1940

DIS - weighted distances to five Boston employment centres

RAD - index of accessibility to radial highways

TAX - full-value property-tax rate per $10,000

PTRATIO - pupil-teacher ratio by town

B - 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town

LSTAT - % lower status of the population

TARGET (value we need to estimate) - Median value of owner-occupied homes in $1000’s

Let's now load the data and do some EDA (Exploratory Data Analysis)

In [ ]:
# loading the data
X, y = load_boston(return_X_y=True)

# feature names
cols = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']

# creating the dataframe for better visualizations and data processing
df = pd.DataFrame(np.c_[X, y], columns=cols + ['TARGET']); df.shape

In [ ]:
df.head()

Let's visualize distribution of each feature using boxplot

We can see that feature scale highly differs, but, fortunately, tree-based models <b>doesn't require</b> feature re-scaling (subtract mean and divide by standard deviation or something similar)

In [ ]:
df.plot(kind='boxb', rot=90, figsize=(15, 5))
plt.grid()

Now let's see correlation (Pearson's correlation) matrix of features and target. 
We can observe that some of the features (e.g LSTAT, RM) are in <b>high correlation</b> with the target, that usually means that they are <b>good predictors</b> for estimating target

In [ ]:
corr = df.corr()
sns.clustermap(corr, figsize=(15, 15), cmap='mako', method='centroid')

In [ ]:
# here we sort features in decreasing order of high correlation with target variable.

np.abs(corr['TARGET']).sort_values(ascending=False)

Now lets move on prediction.

Let's imagine our goal is to optimize MSE (Mean Squred Error).

In that case, trivial solution using constant predictor (always predicting same value irrespective of the input) would be the <b>sample mean</b> of targets in training dataset. One can proof mathematically, that for MSE the optimal constant predictor is indeed a mean. You can google it up :)

Let's find out what the MSE is in this case (for simplicity we don't split the data and do everything on training set for now)

In [ ]:
def mse(x, y):
    return np.mean((y - x) ** 2)

constant_predictor = df['TARGET'].mean()

mse(constant_predictor, df['TARGET'])

We can see empirically that deviations from optimal constant predictor causes MSE to grow, i.e MSE is optimal.

In [ ]:
eps = 1

mse(constant_predictor - eps, df['TARGET']), mse(constant_predictor + eps, df['TARGET'])

Ok, now let's try to do slightly better. 

Let's try to divide our training set in <b>two groups</b> and for each group, calculate <b>their own mean</b>. If we make this split in a smart way, intuitively the two means would be better than using just <b>single global mean</b> as we did in previous example.

One way to do a split is to select on of the features of the data, let's say X, and some threshold, T, and make:<br /><br />
group 1 - { all points having $X<T$ }<br />
group 2 - {all points having $X>=T$}<br />

Then we will have 2 constant predictors - one for each group - and, for every input we first gonna determine its group, and then use group's mean to estimate the target.

This intuition is captured by Decision Tree with depth 1.

In [ ]:
from sklearn.tree import DecisionTreeRegressor, plot_tree

In [ ]:
# defining decision tree with depth 1.
dt = DecisionTreeRegressor(max_depth=1)

# fitting the decision tree on our data/
model = dt.fit(X, y)

# visualizing the tree
plot_tree(model, feature_names=cols, proportion=True)

As we see, decision tree chose a feature "RM" and a threshold T=6.941 to do the split. You can also see the proportion of samples in two groups and value of the mean used for prediction.

Let's verify this picture using python by our own:

In [ ]:
left = df[df['RM'] < 6.941]['TARGET']
right = df[df['RM'] > 6.941]['TARGET']

left.mean(), right.mean(), mse(left.mean(), left), mse(right.mean(), right)

Intuitively, we can go on with splitting and apply same splitting logic recursively on groups (leaf nodes)...

For example, the tree with depth=2 can be seen below

In [ ]:
dt = DecisionTreeRegressor(max_depth=2)
model = dt.fit(X, y)
plot_tree(model, feature_names=cols, proportion=True)

<b>General notes on Decision Trees for Regression</b>

<ul>
    <li>Generally, there is no need to use binary trees (i.e we can have three groups instead of two) and no need to select single feature for splitting. However, the approach we used is most widely used and popular.</li>
    <li>In practice, trees experience <b>low bias and very high variance</b> - i.e they tend to overfit. One of the ways to control tree complexity is to limit it's depth or to set the restriction on the minimal number of datapoints required for node to be split further. Usually, ensembling approaches (follow up in this class) are used to lower the variance.</li>
    <li>For <b>nominal categorical variables</b> (i.e ones that can't be ordered) splitting is done by partitioning the set of values for the variable. For example, if we have X={A, B, C} than we can have {A} {B, C} as a split.</li>
</ul>

Refer here, for more notes: https://scikit-learn.org/stable/modules/tree.html

To be continued...